In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import torch
import os 
import sys 
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm.auto import tqdm
from data import *
from build_batches import load_battery_data

In [ ]:
# load batch with custom function
bat_dict = load_battery_data('/Users/ivan_zorin/Documents/AIRI/data/batt/', is_save=False)

print(bat_dict['b1c0']['cycle_life'])

b1c0_cycles = [int(cycle) for cycle in bat_dict['b1c0']['cycles'].keys()]
print(max(b1c0_cycles))


In [ ]:
# load pickle file of gathered bathches directly with pickle

data_path = '/Users/ivan_zorin/Documents/AIRI/data/batt'
pkl_file = 'batt_data.pkl'

with open(os.path.join(data_path, pkl_file), 'rb') as f:
    bat_data = pickle.load(f)

print(bat_data['b1c0']['cycles'].keys())

temp = [int(cycle) for cycle in bat_data['b1c0']['cycles'].keys()]
print(max(temp))

In [ ]:
data_path = '/Users/ivan_zorin/Documents/AIRI/data/batt/' + 'batt_data.pkl'

In [3]:
dataset = BattData('/Users/ivan_zorin/Documents/AIRI/data/batt/', data_name='batt_data.pkl', is_save_data=False)
print(dataset[0])

data loaded from pickle file
start building dataset
['b1c0', 1, 1087, {'I': array([ 0.        ,  0.21590753,  0.35983085, ..., -0.03422737,
       -0.03422737,  0.        ]), 'Qc': array([0.0000000e+00, 1.0935055e-06, 1.0935055e-06, ..., 1.0710422e+00,
       1.0710422e+00, 1.0710422e+00]), 'Qd': array([0.       , 0.       , 0.       , ..., 1.0706892, 1.0706892,
       1.0706892]), 'Qdlin': array([-3.57431692e-04, -3.18524587e-04, -2.81852414e-04, -2.47206030e-04,
       -2.14376291e-04, -1.83154053e-04, -1.53330172e-04, -1.24695504e-04,
       -9.70409053e-05, -7.01572323e-05, -4.38353411e-05, -1.78660877e-05,
        7.95967154e-06,  3.38510806e-05,  6.00172833e-05,  8.66674235e-05,
        1.14010645e-04,  1.42256092e-04,  1.71612907e-04,  2.02290022e-04,
        2.34405212e-04,  2.67843697e-04,  3.02454149e-04,  3.38085240e-04,
        3.74585640e-04,  4.11804020e-04,  4.49589083e-04,  4.87847990e-04,
        5.26667900e-04,  5.66170505e-04,  6.06477494e-04,  6.47710557e-04,
      

In [8]:
batt_fade = dataset.capacity_fade_approximation()

In [19]:
hi_approximator = PolyApproximator
batt_fade_approx = {}


In [20]:
for batt_name, value in dataset.batt_summary.items():
    QC = value['QC']
    QD = value['QD']

    qc_approx = hi_approximator(degree=2).fit(np.arange(len(QC)), QC)
    qd_approx = hi_approximator(degree=2).fit(np.arange(len(QD)), QD)

    batt_fade_approx[batt_name] = {
                                    'QC': qc_approx,
                                    'QD': qd_approx
                                    }

In [21]:
batt_fade_approx['b1c0']['QC']

{'QC': <data.PolyApproximator at 0x7f98a0a521f0>,
 'QD': <data.PolyApproximator at 0x7f98a0a52f40>}

In [ ]:
dataloader = BattDataloader(dataset, load_features=['I', 'T'], batch_size=1, shuffle=False)
batch = next(iter(dataloader))
print(batch)

In [ ]:
t = tqdm(dataloader)

for i, batch in enumerate(t):
    pass

In [ ]:
for len_ in range(200):
    # print(len_)
    dataset_min_len = len_
    min_idx = []

    for i, sample in enumerate(dataset):
        if dataset_min_len == sample[2]:
            # dataset_min_len = sample[2]
            min_idx += [i]

    if len(min_idx) > 0:
        print(len_)
        print(min_idx)  
        print(len(min_idx))

        check_batts = [dataset[idx][0] for idx in min_idx]
        batch_name = [name[:2] for name in check_batts]

        print(len(batch_name), set(batch_name))
        print('=======')

In [ ]:
data_path = '/Users/ivan_zorin/Documents/AIRI/data/batt'


with open(os.path.join(data_path, 'batch1.pkl'), 'rb') as f:
    bat_data = pickle.load(f)

In [ ]:
len(bat_data.keys())